In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder



# Função para extrair features usando VGG16
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

# Carregar modelo VGG16 pré-treinado
base_model = VGG16(weights='imagenet', include_top=False)

# Diretório raiz do dataset
dataset_dir = '../img/Potato'





In [3]:
X, y = extract_features(dataset_dir, base_model)

PermissionError: [Errno 13] Permission denied: '../img/Potato'

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_encoded, y_test_encoded = encoder_label(y_train, y_test)

NameError: name 'X' is not defined

In [1]:
y_test_encoded

NameError: name 'y_test_encoded' is not defined

In [35]:
y_train_encoded, y_test_encoded = encoder_label(y_train, y_test)

num_models = 15
random_forests = []
decision_trees = []
baggings = []
soft_votes = []

for _ in range(num_models):
    clfRF = RandomForestClassifier(n_estimators=100, random_state=42)
    clfDT = DecisionTreeClassifier(max_depth=4)
    clfB = BaggingClassifier(n_estimators=100, random_state=42)
    eclf = VotingClassifier(estimators=[('rf', clfRF), ('dt', clfDT), ('bagging', clfB)],
                        voting='hard')
    
    clfRF.fit(X_train, y_train_encoded)
    clfDT.fit(X_train, y_train_encoded)
    clfB.fit(X_train, y_train_encoded)
    eclf.fit(X_train, y_train_encoded)
    
    random_forests.append(clfRF)
    decision_trees.append(clfDT)
    baggings.append(clfB)
    soft_votes.append(eclf)

y_preds = []
for clf in soft_votes:
    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)

# Converter as previsões para uma matriz numpy de inteiros
y_preds = np.array(y_preds)

# Votação da maioria
def majority_vote(votes):
    vote_counts = np.bincount(votes)
    return vote_counts.argmax()

# Aplicar votação da maioria ao longo do eixo 0
y_pred_ensemble = np.apply_along_axis(majority_vote, axis=0, arr=y_preds)

# Converter os rótulos de volta para as classes originais
y_pred_ensemble = label_encoder.inverse_transform(y_pred_ensemble)

# Avaliar a acurácia do ensemble
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f'Acurácia do ensemble: {accuracy}')


Acurácia do ensemble: 0.9211136890951276


### Stacking

In [ ]:
num_models = 15
random_forests = []
decision_trees = []
baggings = []
soft_votes = []

In [19]:
# Suponha que você tem uma função para mapear os índices de classe para os nomes das classes
classes = ['saudável', 'doença1', 'doença2']  # Ordem dos rótulos (classes)

# Prever a classe para cada imagem de teste
y_pred_labels = []
for clf in random_forests:
    y_pred = clf.predict(X_test)
    y_pred_labels.append([classes[y] for y in y_pred])

# Exibir resultados de previsão para algumas imagens de teste
num_exemplos = 20
for i in range(num_exemplos):
    print(f'Imagem {i+1}: Predição - {y_pred_labels[0][i]}, {y_pred_labels[1][i]}, {y_pred_labels[2][i]}, {y_pred_labels[3][i]}, {y_pred_labels[4][i]}')

Imagem 1: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 2: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 3: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 4: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 5: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 6: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 7: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 8: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 9: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 10: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 11: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 12: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 13: Predição - saudável, saudável, saudável, saudável, saudável
Imagem 14: Predição - doença1, doença1, doença1, doença1, doença1
Imagem 15: Predição - saudável, saudável, saudável

In [20]:
def encoder_label(y_train, y_test):
    # Codificar rótulos como inteiros
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    return y_train_encoded, y_test_encoded

In [22]:
def features_and_targets(dataset_dir):
    # Lista para armazenar features e rótulos
    X = []
    y = []
    # Iterar sobre as pastas de classes (saudável, doença1, doença2)
    for classe in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, classe)
        if os.path.isdir(class_dir):
            # Iterar sobre os arquivos de imagens dentro de cada classe
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                # Extrair features usando VGG16
                features = extract_features(img_path, base_model)
                X.append(features)
                y.append(classe)  # A classe é o nome da subpasta (saudável, doença1, doença2)
    
    # Converter para arrays numpy
    X = np.array(X)
    y = np.array(y)
    

    return X, y